In [1]:
import pandas as pd

In [107]:
def open_csv(fname):
    df = pd.read_csv(fname)
    try: 
        df.drop('type_name', axis=1, inplace=True)
    except:
        print('Error with dropping type_name column')
        
    if fname == 'statistics - dem_age-structure.csv':
        df.fillna(value=0, inplace=True)
    
    try:
        df['date_stamp'] = pd.to_datetime(df['date_stamp'], format='%Y-%m-%d')
    except:
        print('Error with datetime')
    
    for col in list(df):
        if col not in ['name', 'date_stamp']:
            try:
                df[col] = df[col].astype(int)
            except:
                continue
        
    return df

In [108]:
empl = open_csv('statistics - dem_employment.csv')
age = open_csv('statistics - dem_age-structure.csv')

In [96]:
#issue with age pyramid -- check numbers in spreadsheets first
def age_check(df):
    col_list = [col for col in list(df) if col.startswith('age') and col != 'age_general']
    #df2 = pd.DataFrame({'summed': df[col_list].sum(axis=1), 'age_general': df['age_general']})
    df_copy = df.set_index('name').copy()
    diff = df_copy[col_list].sum(axis=1) - df_copy['age_general']
    diff_cols = diff[diff != 0]
    if len(diff_cols) != 0:
        print('These regions still dont add up:    ' + str(diff_cols.index.values.tolist()))
    else:
        print('All good with age data!')
age_check(age)

These regions still dont add up:    ['Радивонівська', "Мар'янівка", 'Степанівка', 'Пушкареве', 'Семенівка']


In [121]:
e = empl.copy()
e.set_index(['name', 'date_stamp']).T

name,Якимівська,Степанівська,Багачанська,Радивонівська,Мала Решетилівка,Байрак,Буряківщина,Бутова Долина,Гарнокут,Довгалівка,...,Багачка Перша,Семенівка,Широке,Радивонівка,Мар'янівка,Володимирівка,Велика Багачка,Якимове,Балюки,Затон
date_stamp,2017-01-01,2017-01-01,2017-01-01,2017-01-01,2017-01-01,2017-01-01,2017-01-01,2017-01-01,2017-01-01,2017-01-01,...,2017-01-01,2017-01-01,2017-01-01,2017-01-01,2017-01-01,2017-01-01,2017-01-01,2017-01-01,2017-01-01,2017-01-01
agriculture,30.0,107.0,103.0,381.0,20.0,126.0,7.0,94.0,5.0,176.0,...,23.0,30.0,20.0,274.0,98.0,3.0,965.0,20.0,10.0,19.0
industry,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,55.0,3.0,1.0,NaN
construction,2.0,NaN,15.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,...,3.0,NaN,3.0,NaN,NaN,NaN,67.0,2.0,NaN,NaN
service,15.0,3.0,NaN,4.0,NaN,11.0,NaN,6.0,NaN,3.0,...,NaN,NaN,NaN,2.0,2.0,NaN,325.0,10.0,5.0,1.0
tourism,4.0,5.0,NaN,27.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,19.0,8.0,NaN,79.0,3.0,1.0,NaN
trans_comm,8.0,3.0,3.0,6.0,NaN,NaN,NaN,2.0,NaN,1.0,...,NaN,NaN,1.0,4.0,2.0,NaN,55.0,6.0,2.0,1.0
finance,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,10.0,4.0,NaN
real_estate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
government,5.0,5.0,5.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,12.0,NaN,7.0,NaN,NaN,213.0,4.0,1.0,NaN


In [76]:
"agriculture industry construction service tourism trans_comm finance real_estate government education health culture other".split()

['agriculture',
 'industry',
 'construction',
 'service',
 'tourism',
 'trans_comm',
 'finance',
 'real_estate',
 'government',
 'education',
 'health',
 'culture',
 'other']